In [2]:
%load_ext sql
%sql sqlite:///PS1.db

Problem Set 1 [75 points]
=======

### Deliverables:

Submit your queries (and only those) using the `submission_template.txt` file that is posted on the class website. Follow the instructions on the file! Upload the file at Canvas (under PS1).


### Instructions / Notes:

* Run the top cell above to load the database `PS1.db` (make sure the database file, `PS1.db`, is in the same directory as this IPython notebook is running in)
* Some of the problems involve _changing_ this database (e.g. deleting rows)- you can always re-download `PS1.db` or make a copy if you want to start fresh!
* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that your final answer for each question is _in its own cell_ and _clearly indicated_**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: To restart the SQL connection, you must restart the entire python kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute the sql connection cell at top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* _Have fun!_

Problem 1: Linear Algebra [25 points]
------------------------

Two random 3x3 ($N=3$) matrices have been provided in tables `A` and `B`, having the following schema:
> * `i INT`:   Row index
> * `j INT`:   Column index
> * `val INT`: Cell value

**Note: all of your answers below _must_ work for any _square_ matrix sizes, i.e. any value of $N$**.

Note how the matrices are represented - why do we choose this format?  Run the following queries to see the matrices in a nice format:

In [3]:
%sql SELECT group_concat(val, " , ") AS "A" FROM A GROUP BY i;

 * sqlite:///PS1.db
Done.


A
"7 , 5 , 8"
"10 , 7 , 7"
"2 , 0 , 5"


In [4]:
%sql SELECT group_concat(val, " , ") AS "B" FROM B GROUP BY i;

 * sqlite:///PS1.db
Done.


B
"9 , 6 , 10"
"7 , 6 , 9"
"1 , 1 , 7"


### Part (a): Matrix addition [5 points]

The sum of a matrix $A$ (having dimensions $n\times m$) and a matrix $B$ (having dimensions $n\times m$) is the matrix $C$ (of dimension $n\times m$) having cell at row $i$ and column $j$ equal to:

$C_{ij} = A_{ij} + B_{ij}$

Write a single SQL query to get the sum of $A$ and $B$ (in the same format as $A$ and $B$):

In [5]:
%%sql 
SELECT A.i AS i, B.j AS j, (A.val+B.val) AS val
FROM A
JOIN B
WHERE A.i == B.i AND A.j == B.j

 * sqlite:///PS1.db
Done.


i,j,val
0,0,16
0,1,11
0,2,18
1,0,17
1,1,13
1,2,16
2,0,3
2,1,1
2,2,12


### Part (b): Dot product [5 points]

The _dot product_ of two vectors

$a = \begin{bmatrix}a_1 & a_2 & \dots & a_n\end{bmatrix}$

and

$b = \begin{bmatrix}b_1 & b_2 & \dots & b_n\end{bmatrix}$

is

$a\cdot b = \sum_{i=1}^n a_ib_i = a_1b_1 + a_2b_2 + \dots + a_nb_n$

Write a _single SQL query_ to take the dot product of the **third column of $A$** and the **second column of $B$.**:

In [6]:
%%sql
SELECT SUM(A*B) AS DotProduct
FROM(SELECT A,B
	 FROM (SELECT val AS A,i,j FROM A WHERE j == 2) AS A
	 JOIN (SELECT val AS B,i AS B_i,j AS B_j FROM B WHERE j == 1)
 	 WHERE A.i == B_i
	 GROUP BY A.i
	)
;

 * sqlite:///PS1.db
Done.


DotProduct
95


### Part (c): Matrix multiplication [5 points]

The product of a matrix $A$ (having dimensions $n\times m$) and a matrix $B$ (having dimensions $m\times p$) is the matrix $C$ (of dimension $n\times p$) having cell at row $i$ and column $j$ equal to:

$C_{ij} = \sum_{k=1}^m A_{ik}B_{kj}$

In other words, to multiply two matrices, get each cell of the resulting matrix $C$, $C_{ij}$, by taking the _dot product_ of the $i$th row of $A$ and the $j$th column of $B$.

Write a single SQL query to get the matrix product of $A$ and $B$ (in the same format as $A$ and $B$):

In [7]:
%%sql
SELECT A.i AS i, B.j AS j, SUM(A.val*B.val) as val
FROM A
INNER JOIN B
ON A.j == B.i
GROUP BY A.i, B.j;

 * sqlite:///PS1.db
Done.


i,j,val
0,0,106
0,1,80
0,2,171
1,0,146
1,1,109
1,2,212
2,0,23
2,1,17
2,2,55


### Part (d): Matrix power [10 points]

The power $A^n$ of a matrix $A$ is defined as the matrix product of $n$ copies of $A$. 

Write a _single SQL query_ that computes the **fourth power** of matrix $A$, in other words, $A^4 = A \cdot A \cdot A \cdot A$:

In [8]:
%%sql
SELECT A.i AS i, B.j AS j, SUM(A.val*B.val) AS val
FROM A
INNER JOIN (SELECT A.i AS i, B.j AS j, SUM(A.val*B.val) AS val
		  	FROM A
			INNER JOIN (SELECT A.i AS i, B.j AS j, SUM(A.val*B.val) AS val
						FROM A
						INNER JOIN (SELECT * FROM A) AS B
						ON A.j == B.i
						GROUP BY A.i, B.j) AS B
			ON A.j == B.i
			GROUP BY A.i, B.j
		) AS B
ON A.j == B.i
GROUP BY A.i,B.j;

 * sqlite:///PS1.db
Done.


i,j,val
0,0,27149
0,1,16290
0,2,31916
1,0,36892
1,1,22221
1,2,43134
2,0,5284
2,1,3080
2,2,6465


Problem 2: The Sales Database [25 points]
----------------------------------------------

We've prepared and loaded a dataset related to sales data from a company. The dataset has the following schema:

> `Holidays (WeekDate, IsHoliday)`

> `Stores (Store, Type, Size)`

> `TemporalData(Store, WeekDate, Temperature, FuelPrice, CPI, UnemploymentRate)`

> `Sales (Store, Dept, WeekDate, WeeklySales)`

Before you start writing queries on the database, find the schema and the constraints (keys, foreign keys). 

### Part (a): Sales during Holidays [10 points]

Using a _single SQL query_, find the store(s) with the largest overall sales during holiday weeks. Further requirements:
* Use the `WITH` clause before the main body of the query to compute a subquery if necessary.
* Return a relation with schema `(Store, AllSales)`.

Write your query here:

In [152]:
%%sql
WITH HolidaySales AS (SELECT Store, SUM(WeeklySales) AS AllSales
					  FROM Sales JOIN Holidays
					  WHERE Sales.WeekDate == Holidays.WeekDate AND IsHoliday == "TRUE"
					  GROUP BY Store)
SELECT Store, AllSales
FROM HolidaySales
ORDER BY AllSales DESC
LIMIT 1;

 * sqlite:///PS1.db
Done.


Store,AllSales
20,22490350.81000001


### Part (b): When Holidays do not help Sales [10 points]

Using a _single SQL query_, compute the **number** of non-holiday weeks that had larger sales than the overall average sales during holiday weeks. Further requirements:
* Use the `WITH` clause before the main body of the query to compute a subquery if necessary.
* Return a relation with schema `(NumNonHolidays)`.

Write your query here:

In [212]:
%%sql 
WITH HolidaySales AS (SELECT Sales.WeekDate, SUM(WeeklySales) AS TotalSales
					  FROM Sales, Holidays
					  WHERE Sales.WeekDate == Holidays.WeekDate AND Holidays.IsHoliday == "TRUE"
					  GROUP BY Sales.WeekDate),
	 HolidayAvg AS (SELECT AVG(TotalSales) AS avg
	 				FROM HolidaySales),
	 NonHolidaySales AS (SELECT * FROM Sales, Holidays, HolidayAvg
						 WHERE IsHoliday == "FALSE" AND Sales.WeekDate == Holidays.WeekDate
						 GROUP BY Sales.WeekDate
						 HAVING SUM(Sales.WeeklySales) > avg)
SELECT COUNT(WeekDate) AS NumNonHolidays FROM NonHolidaySales;

 * sqlite:///PS1.db
Done.


NumNonHolidays
8


### Part (c): Total Summer Sales [5 points]

Using a _single SQL query_, compute the total sales during summer (months 6,7,and 8) for each type of store. Further requirements:
* Return a relation with schema `(type, TotalSales)`.

*Hint:* SQLite3 does not support native operations on the DATE datatype. To create a workaround, you can use the `LIKE` predicate and the string concatenation operator (||). You can also use the substring operator that SQLite3 supports (`substr`).

Write your query here:

In [228]:
%%sql
SELECT Stores.Type, SUM(WeeklySales) AS TotalSales
FROM Sales, Stores
WHERE (WeekDate LIKE '%-06-%'
OR WeekDate LIKE '%-07-%'
OR WeekDate LIKE '%-08-%')
AND Sales.Store == Stores.Store
GROUP BY Stores.Type;

 * sqlite:///PS1.db
Done.


type,TotalSales
A,1211554899.8500097
B,561610722.3999932
C,112555450.65999933


Problem 3: The Traveling SQL Server Salesman Problem [25 points]
--------------------------------------------------

SQL Server salespeople are lucky as far as traveling salespeople go- they only have to sell one or two big enterprise contracts, at one or two offices in Wisconsin, in order to make their monthly quota!

Answer the following questions using the table of streets connecting company office buildings.

**Note that for convenience all streets are included _twice_, as $A \rightarrow B$ and $B \rightarrow A$.  This should make some parts of the problem easier, but remember to take it into account!**

In [12]:
%sql SELECT * FROM streets LIMIT 4;

 * sqlite:///PS1.db
Done.


id,direction,A,B,d
0,F,UW-Madison,DooHickey Collective,7
0,R,DooHickey Collective,UW-Madison,7
1,F,DooHickey Collective,Gizmo Corp,2
1,R,Gizmo Corp,DooHickey Collective,2


### Part (a): One-hop, two-hop, three-hop... [10 points]

Our salesperson has stopped at UW-Madison, to steal some cool new RDBMS technology from CS564-ers, and now wants to go sell it to a company _within 9 miles of UW-Madison and _passing through no more than 3 distinct streets_.  Write a single query, not using `WITH` (see later on), to find all such companies.

Your query should return the schema `(company, distance)` where distance is cumulative from UW-Madison.

Write your query here:

In [379]:
%%sql
DROP VIEW IF EXISTS farm_stop;
CREATE VIEW farm_stop AS
SELECT A, B, d AS distance, 10*d AS id
FROM streets
WHERE direction == "F"
UNION SELECT s1.A, s2.B, (s1.d + s2.d) AS distance, (10*s1.d + 7*s2.d) AS id
	  FROM streets AS s1, streets AS s2
	  WHERE s1.B == s2.A 
	  AND s1.direction == "F" AND s2.direction == "F"
UNION SELECT s1.A, s3.B, (s1.d + s2.d + s3.d) AS distance, (10*s1.d + 7*s2.d + 3*s3.d) AS id
	  FROM streets AS s1, streets AS s2, streets AS s3
	  WHERE s1.B == s2.A AND s2.B == s3.A AND s1.A != s3.B
	  AND s1.direction == "F" AND s2.direction == "F" AND s3.direction == "F";

SELECT f1.B AS company_1, f2.B AS company_2, (f1.distance + f2.distance) AS distance
FROM farm_stop AS f1, farm_stop AS f2
WHERE f1.A == "UW-Madison" AND f2.A == "UW-Madison" AND f1.B != f2.B
	  AND (f1.distance + f2.distance) <= 15
GROUP BY f1.id + f2.id;


 * sqlite:///PS1.db
Done.
Done.
Done.


company_1,company_2,distance
DooHickey Collective,Gadget Corp,13
DooHickey Corp,Gadget Corp,15
Gadget Corp,Gizmo Corp,15
Gadget Collective,Gadget Corp,15


### Part (b): A stop at the Farm [10 points]

Now, our salesperson is out in the field, and wants to see all routes- and their distances- which will take him/her from a company $A$ to a company $B$, with the following constraints:
* The route must pass through UW-Madison (in order to pick up new RDBMS tech to sell!)
* $A$ and $B$ must _each individually_ be within 2 hops of UW-Madison
* $A$ and $B$ must be different companies
* _The total distance must be $<= 15$_
* Do not use `WITH`
* If you return a path $A \rightarrow B$, _do not include_ $B \rightarrow A$ in your answer!

In order to make your answer a bit cleaner, you may split into two queries, one of which creates a `VIEW`.  A view is a virtual table based on the output set of a SQL query.  A view can be used just like a normal table- the only difference under the hood is that the DBMS re-evaluates the query used to generate it each time a view is queried by a user (thus the data is always up-to date!)

Here's a simple example of a view:

In [14]:
%%sql 
DROP VIEW IF EXISTS short_streets;
CREATE VIEW short_streets AS 
SELECT A, B, d FROM streets WHERE d < 3;
SELECT * FROM short_streets LIMIT 3;

 * sqlite:///PS1.db
Done.
Done.
Done.


A,B,d
DooHickey Collective,Gizmo Corp,2
Gizmo Corp,DooHickey Collective,2
Gizmo Corp,Widget Industries,1


Write your query or queries here:

In [377]:
%%sql
DROP VIEW IF EXISTS farm_stop;
CREATE VIEW farm_stop AS
SELECT A, B, d AS distance, 10*d AS id
FROM streets
WHERE direction == "F"
UNION SELECT s1.A, s2.B, (s1.d + s2.d) AS distance, (10*s1.d + 7*s2.d) AS id
	  FROM streets AS s1, streets AS s2
	  WHERE s1.B == s2.A 
	  AND s1.direction == "F" AND s2.direction == "F"
UNION SELECT s1.A, s3.B, (s1.d + s2.d + s3.d) AS distance, (10*s1.d + 7*s2.d + 3*s3.d) AS id
	  FROM streets AS s1, streets AS s2, streets AS s3
	  WHERE s1.B == s2.A AND s2.B == s3.A AND s1.A != s3.B
	  AND s1.direction == "F" AND s2.direction == "F" AND s3.direction == "F";

SELECT f1.B AS company_1, f2.B AS company_2, (f1.distance + f2.distance) AS distance
FROM farm_stop AS f1, farm_stop AS f2
WHERE f1.A == "UW-Madison" AND f2.A == "UW-Madison" AND f1.B != f2.B
	  AND (f1.distance + f2.distance) <= 15
GROUP BY f1.id + f2.id;


 * sqlite:///PS1.db
Done.
Done.
Done.


company_1,company_2,distance
DooHickey Collective,Gadget Corp,13
DooHickey Corp,Gadget Corp,15
Gadget Corp,Gizmo Corp,15
Gadget Collective,Gadget Corp,15


### Part (c): Finding Triangles [5 points]

Finally, our salesperson wants to find a route that goes from company $A$ to company $B$ to company $C$ and then back to company $A$ with the following constraints:
* $A$, $B$, $C$ must be different companies
* Do not use `WITH` 
* Output each such route that you find once (use the id's as a way to break ties)
* Output the distance of the route

Write your query here:

In [374]:
%%sql
SELECT s1.A AS A, s2.A AS B, s3.A AS C, (s1.d + s2.d + s3.d) AS distance
FROM streets AS s1, streets AS s2, streets AS s3
WHERE s1.B == s2.A AND s2.B == s3.A AND s1.direction == "F" AND s2.direction == "F" AND s3.direction == "F" AND s3.B == s1.A
GROUP BY distance
HAVING MIN(3*s1.d + 4*s2.d + 5*s3.d);

 * sqlite:///PS1.db
Done.


A,B,C,distance
Widget Collective,GadgetCo,Thing Industries,18
